In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
df = spark.read.format("parquet").load("abfss://bronze@stretaildataproj.dfs.core.windows.net/orders")

In [0]:
df.display()
df.printSchema()

In [0]:
df=df.withColumn("order_date", col("order_date").cast("date"))\
    .withColumn("total_amount" , col("total_amount").cast("double"))\
    .withColumn("quantity" , col("quantity").cast("integer"))
df.printSchema()

In [0]:
df = df.drop("_rescued_data")
df.display()

In [0]:
df = df.withColumn("order_date", to_timestamp(col("order_date")))
df.display()
df.printSchema()
